In [1]:
%load_ext autoreload
%autoreload 2
%pdb
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger().setLevel(logging.INFO)

Automatic pdb calling has been turned ON


In [2]:
import sys
sys.path.insert(0, '/home/ubuntu/Documents/infembed')
#sys.path.insert(0, '/home/ubuntu/Documents/infembed/infembed')
# sys.path.insert(0, '/home/ubuntu/Documents/infembed/infembed')
# sys.path.insert(0, '/home/ubuntu/Documents/infembed/data')
from data._core.spotcheck import get_spotcheck_dataloader, get_blindspots_df
# sys.path.insert(0, '/home/ubuntu/Documents/infembed/models')
#sys.path.insert(0, '/home/ubuntu/Documents/infembed/')
# sys.path.insert(0, )
from infembed.embedder._core.fast_kfac_embedder import FastKFACEmbedder
import torchvision
from torch.utils.data import Subset, DataLoader, default_collate, Dataset
from torchvision.models import ResNet18_Weights, resnet18
import torch.nn as nn
from infembed.clusterer._core.sklearn_clusterer import SklearnClusterer
from infembed.clusterer._core.faiss_clusterer import FAISSClusterer
from infembed.clusterer._core.rule_clusterer import RuleClusterer
from sklearn.cluster import KMeans
from tqdm import tqdm
import pandas as pd
import torch
torch.multiprocessing.set_start_method('spawn')
from typing import List
from infembed.utils.common import Data
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Callable
from torch import Tensor
from typing import Tuple
from models._utils.common import init_linear
from lightning.pytorch.callbacks import EarlyStopping
import lightning.pytorch as pl
import lightning as L
import torch.nn.functional as F
import numpy as np
import pandas as pd
from data._utils.common import LimitIterableDataset
from data._core.tinystories import TinyStoriesCollateFn, TinyStoriesDataset
from data._utils.common import IterableDatasetToDataset

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.


### figure out device to compute embeddings on ###

In [3]:
DEVICE = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print('device:', DEVICE)

device: cuda:0


### define data
We will define the following:
- `eval_dataset`: `Dataset` for evaluation data.  This is used to retrieve individual examples for displaying.  This should be the same data for which the embeddings we load later were computed
- `eval_dataloader`: `DataLoader` for evaluation data.  This is used to compute metadata for the evaluation data

In [5]:
eval_dataset = IterableDatasetToDataset(
    LimitIterableDataset(
        dataset=TinyStoriesDataset(
            "/home/ubuntu/Documents/infembed/files/tinystories/TinyStoriesV2-GPT4-valid.txt"
        ),
    ),
)

eval_dataloader = DataLoader(
    dataset=eval_dataset,
    collate_fn=TinyStoriesCollateFn(duplicate=True, device=DEVICE),
    batch_size=4,
)

### define model ###
only used to get metadata

In [7]:
from models._utils.common import HuggingfaceWrapperModel, load_model
from transformers import AutoModelForCausalLM

model = load_model(
    model=HuggingfaceWrapperModel(
        model=AutoModelForCausalLM.from_pretrained(
            pretrained_model_name_or_path='roneneldan/TinyStories-33M',
        )
    ),
    device=DEVICE,
    eval=True,
)
model

HuggingfaceWrapperModel(
  (model): GPTNeoForCausalLM(
    (transformer): GPTNeoModel(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(2048, 768)
      (drop): Dropout(p=0.0, inplace=False)
      (h): ModuleList(
        (0-3): 4 x GPTNeoBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPTNeoAttention(
            (attention): GPTNeoSelfAttention(
              (attn_dropout): Dropout(p=0.0, inplace=False)
              (resid_dropout): Dropout(p=0.0, inplace=False)
              (k_proj): Linear(in_features=768, out_features=768, bias=False)
              (v_proj): Linear(in_features=768, out_features=768, bias=False)
              (q_proj): Linear(in_features=768, out_features=768, bias=False)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPTNeoMLP(
            (c_fc): Linear(in_fe

### load embeddings for evaluation data ###

In [8]:
embeddings_path = '/home/ubuntu/Documents/infembed/examples/tinystories/hydra_outputs/run_embedder/last_layer/dim=500/blocks=5/embeddings.pt'
embeddings = torch.load(open(embeddings_path, 'rb'))
embeddings.shape

torch.Size([27630, 479])

optionally, we can only keep some elements of the embedding (relevant for `ArnoldiEmbedder`)

In [9]:
DIMS_TO_KEEP = None
if DIMS_TO_KEEP is not None:
    # embeddings = embeddings[:, -DIMS_TO_KEEP:]
    embeddings = embeddings[:, :DIMS_TO_KEEP]

optionally, we can normalize the embeddings to have the same norm

In [10]:
NORMALIZE = True
if NORMALIZE:
    embeddings = embeddings / torch.linalg.norm(embeddings, dim=1)[:, None]

In [11]:
if True:
    # already assume embeddings fit in memory, so no point in using incremental methods
    pass

we then package them into a `Data` instance, which contains all kinds of data that could possibly be used to do the subsequent clustering, i.e. including tabular metadata as well.

In [11]:
data = Data(embeddings=embeddings)

### compute metadata for evaluation data ###
this will be the ingredient needed to display the clusters.  later on, it will also be used by the rule-based clusterer.  therefore, we also add it to the running `Data` instance for easy access.

In [50]:
def _get_predictions_and_labels(_model, dataloader):
    dfs = []
    for batch in tqdm(dataloader):
        prediction_prob = (
            torch.nn.functional.softmax(_model(*batch[:-1]), dim=1)
            .detach()
            .to(device="cpu")
        )
        prediction_label = torch.argmax(prediction_prob, dim=1).to(device="cpu")
        label = batch[-1].to(
            device="cpu"
        )  # assuming batch is a tensor.  if not, can check
        dfs.append(
            pd.DataFrame(
                {
                    "prediction_label": prediction_label,
                    "label": label,
                    "prediction_prob": list(prediction_prob.numpy()),
                }
            )
        )
    df = pd.concat(dfs, axis=0)
    df.index = list(range(len(df)))
    return df

if False:
    metadata = _get_predictions_and_labels(model, eval_dataloader)

100%|██████████| 1563/1563 [07:17<00:00,  3.57it/s]


read in predictions

In [38]:
if False:
    import glob
    preds_path = "/home/ubuntu/Documents/infembed/examples/imagenet/hydra_outputs/lightning_predict/val/preds"
    preds_paths = glob.glob(f"{preds_path}/*")
    preds_paths = sorted(preds_paths, key=lambda path: int(path.split('/')[-1].split('.')[0]))
    preds = pd.concat([pd.read_csv(path, index_col=0) for path in preds_paths], axis=0)
    preds.columns = list(range(preds.shape[1]))
    prediction_label = preds.idxmax(axis=1)
    prediction_label.index = list(range(len(prediction_label)))
    prediction_label

0        817
1        817
2        817
3        817
4        817
        ... 
49995    817
49996    817
49997     92
49998    817
49999    817
Length: 50000, dtype: int64

get labels

In [15]:
if False:
    label = pd.Series(torch.cat([batch[-1] for batch in eval_dataloader], dim=0).cpu().numpy())

create metadata

In [22]:
if False:
    metadata = pd.DataFrame({'prediction_label': prediction_label, 'label': label})
    metadata

,prediction_label,label
0,817,0
1,817,0
2,817,0
3,817,0
4,817,0
...,...,...
49995,817,999
49996,817,999
49997,92,999
49998,817,999


In [12]:
# data.metadata = metadata
data.dataset = eval_dataset

### define clusterer ###

In [13]:
if False:
    clusterer = SklearnClusterer(sklearn_clusterer=KMeans(n_clusters=25))
if True:
    clusterer = FAISSClusterer(k=25, spherical=True)

### do the clustering ###

In [14]:
clusters = clusterer.fit_predict(data)

### define ways to display clusters ###
these will all be functions whose input is a list of list of indices in the evaluation dataset

In [56]:
from infembed.visualization._core.common import (
    DisplayMetadata,
    DisplayPIL,
    DisplayPredictionAndLabels,
    DisplaySingleExamples,
    PerClusterDisplayer,
    DisplayAccuracy,
)

class PrintDisplay(SingleExampleDisplayer):
    def __call__(self, i, data):
        print(data[i])


displayers = [
    PerClusterDisplayer(
        [
            DisplayAccuracy(prediction_col="prediction_label", label_col="label"),
            DisplayPredictionAndLabels(
                prediction_col="prediction_label_name", label_col="label_name"
            ),
            DisplaySingleExamples(
                [
                    DisplayMetadata(["label_name", "prediction_label_name"]),
                    DisplayPIL(),
                ],
                limit=3,
            ),
        ]
    )
]

### display the clusters ###

In [57]:
for displayer in displayers:
    displayer(clusters, data)

cluster #0
accuracy: 0.65 (1245/1926)
prediction: {'albatross': 47, 'goldfinch': 45, 'cardoon': 44, 'liner': 43, 'West Highland white terrier': 43, 'spotted salamander': 42, 'toucan': 42, 'running shoe': 42, 'Italian greyhound': 41, 'mailbox': 40, 'whiptail': 40, 'king snake': 39, 'submarine': 38, 'Newfoundland': 38, 'padlock': 37, 'shower cap': 37, 'flatworm': 37, 'cello': 36, 'cash machine': 34, 'cab': 34, 'computer keyboard': 34, 'cockroach': 33, 'bookcase': 32, 'mitten': 32, 'isopod': 32, 'gown': 31, 'monitor': 29, 'scale': 29, 'trombone': 28, 'vine snake': 26, 'alp': 25, 'soup bowl': 24, 'lighter': 24, 'desktop computer': 23, 'screen': 22, 'passenger car': 22, 'plunger': 21, 'mouse': 18, 'bulletproof vest': 17, 'overskirt': 14, 'valley': 11, 'desk': 11, 'hoopskirt': 11, 'groom': 8, 'television': 8, 'sea slug': 8, 'apron': 7, 'cornet': 7, 'sarong': 6, 'electric locomotive': 6, 'military uniform': 6, 'entertainment center': 6, 'green snake': 5, 'walking stick': 5, 'minibus': 5, 'mix

### define rule clusterer ###

In [61]:
def _accuracy(data):
    return (data.metadata["prediction_label"] == data.metadata["label"]).mean()


def _size(data):
    return len(data)


rule_clusterer = RuleClusterer(
    # clusterer_getter=lambda n_clusters: SklearnClusterer(KMeans(n_clusters=n_clusters)),
    clusterer_getter=lambda n_clusters: FAISSClusterer(k=n_clusters, spherical=True),
    cluster_rule=lambda data: _accuracy(data) < 0.2 and _size(data) >= 10,
    stopping_rule=lambda data: _size(data) < 50,
    max_depth=7,
    branching_factor=3,
)

### do the rule clustering ###

In [62]:
rule_clusters = rule_clusterer.fit_predict(data)

WARNING clustering 104 points to 3 centroids: please provide at least 117 training points
WARNING clustering 51 points to 3 centroids: please provide at least 117 training points
WARNING clustering 51 points to 3 centroids: please provide at least 117 training points
WARNING clustering 82 points to 3 centroids: please provide at least 117 training points
WARNING clustering 50 points to 3 centroids: please provide at least 117 training points
WARNING clustering 50 points to 3 centroids: please provide at least 117 training points
WARNING clustering 53 points to 3 centroids: please provide at least 117 training points
WARNING clustering 51 points to 3 centroids: please provide at least 117 training points
WARNING clustering 105 points to 3 centroids: please provide at least 117 training points
WARNING clustering 50 points to 3 centroids: please provide at least 117 training points
WARNING clustering 95 points to 3 centroids: please provide at least 117 training points
WARNING clustering 

### display the rule clusters

In [63]:
for displayer in displayers:
    displayer(rule_clusters, data)

cluster #0
accuracy: 0.17 (3/18)
prediction: {'beagle': 7, 'Walker hound': 4, 'English foxhound': 3, 'Airedale': 1, 'basenji': 1, 'Brittany spaniel': 1, 'bluetick': 1}
label: {'English foxhound': 18}
cluster #1
accuracy: 0.09 (1/11)
prediction: {'tabby': 5, 'tiger': 2, 'sleeping bag': 1, 'black-footed ferret': 1, 'tiger cat': 1, 'Siamese cat': 1}
label: {'tiger cat': 11}
cluster #2
accuracy: 0.18 (9/50)
prediction: {'hair spray': 9, 'lotion': 8, 'sunscreen': 2, 'web site': 2, 'violin': 1, 'hand blower': 1, 'barbershop': 1, 'seat belt': 1, 'oil filter': 1, 'water bottle': 1, 'lipstick': 1, 'teapot': 1, 'bathtub': 1, 'swing': 1, 'stethoscope': 1, 'hourglass': 1, 'bow tie': 1, 'joystick': 1, 'drumstick': 1, 'Band Aid': 1, 'dough': 1, "carpenter's kit": 1, 'pill bottle': 1, 'kimono': 1, 'groom': 1, 'soap dispenser': 1, 'cornet': 1, 'grand piano': 1, 'vacuum': 1, 'golf ball': 1, 'punching bag': 1, 'switch': 1, 'wig': 1}
label: {'hair spray': 50}
cluster #3
accuracy: 0.18 (9/50)
prediction: 